In [ ]:
# -----------------------------------------------------
# Guia Python completo para análises geoespaciais com PostgreSQL/PostGIS
# e bibliotecas populares no Google Colab (usuários iniciantes)
# -----------------------------------------------------

# ==== 1) Instalar bibliotecas essenciais ====
# Cole este comando em uma célula e execute
!pip install --quiet \
    pandas geopandas shapely matplotlib contextily fiona rasterio \
    sqlalchemy psycopg2-binary geoalchemy2

# ==== 2) Importar pacotes Python ====
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, Polygon
import matplotlib.pyplot as plt
import contextily as ctx
import rasterio
from sqlalchemy import create_engine, text
from geoalchemy2 import Geometry, WKTElement

# ==== 3) Configurar conexão com PostgreSQL/PostGIS ====
# Substitua USER, PASSWORD, HOST, PORT e DBNAME
db_user = 'USER'
db_password = 'PASSWORD'
db_host = 'HOST'
db_port = '5432'
db_name = 'DBNAME'

database_url = (
    f"postgresql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}"
)
en = create_engine(database_url)
print("Conectado a:", en)

# ==== 4) Criar schema e tabela de área de estudo ====
with en.begin() as conn:
    conn.execute(text("CREATE SCHEMA IF NOT EXISTS deltamap;"))
    conn.execute(text(
        """
        CREATE TABLE IF NOT EXISTS deltamap.area_estudo (
            id SERIAL PRIMARY KEY,
            nome TEXT,
            geom geometry(Polygon, 4674)
        );
        """
    ))
    print("Schema e tabela criados/verificados.")

# ==== 5) Inserir e consultar geometria de exemplo ====
# Inserir polígono WKT
wkt_poly = 'POLYGON((-50.0 0.5, -49.0 0.5, -49.0 -0.5, -50.0 -0.5, -50.0 0.5))'
geom = WKTElement(wkt_poly, srid=4674)
df = pd.DataFrame([{'nome': 'Delta Norte', 'geom': geom}])
df.to_sql(
    name='area_estudo', con=en, schema='deltamap', if_exists='append', index=False,
    dtype={'geom': Geometry('POLYGON', srid=4674)}
)
print("Registro inserido.")

# Carregar em GeoDataFrame
gdf_area = gpd.GeoDataFrame.from_postgis(
    "SELECT id, nome, geom FROM deltamap.area_estudo;",
    con=en, geom_col='geom'
)
print(gdf_area)

# ==== 6) Exemplo de análise geoespacial local (no Colab) ====
# Criar buffer de 10 km em torno da área de estudo
# Convertendo para CRS métrico (UTM) para medições precisas
gdf_area_utm = gdf_area.to_crs(epsg=3857)
gdf_buffer = gdf_area_utm.copy()
gdf_buffer['geometry'] = gdf_area_utm.buffer(10000)  # 10 km

# Plotar área + buffer
ax = gdf_area_utm.plot(edgecolor='blue', facecolor='none', linewidth=2)
gdf_buffer.plot(ax=ax, edgecolor='red', facecolor='none', linestyle='--')
ctx.add_basemap(ax, source=ctx.providers.Stamen.Terrain)
ax.set_title('Área de Estudo e Buffer de 10 km')
ax.set_axis_off()
plt.show()

# ==== 7) Importar shapefile e executar spatial join ====
# Substitua '/path/to/data/ucs.shp'
ucs = gpd.read_file('/path/to/data/ucs.shp')
ucs = ucs.to_crs(gdf_area.crs)
joined = gpd.sjoin(ucs, gdf_area, how='inner', predicate='intersects')
print(f"Número de UCs intersectando a área: {len(joined)}")

# ==== 8) Calcular estatísticas de área ====
joined['area_uc_km2'] = joined.area / 1e6
print(joined[['nome_uc', 'area_uc_km2']])

# ==== 9) Exportar resultado para CSV e GeoJSON ====
joined[['nome_uc', 'area_uc_km2']].to_csv('ucs_intersect.csv', index=False)
joined.to_file('ucs_intersect.geojson', driver='GeoJSON')
print("Resultados exportados: ucs_intersect.csv, ucs_intersect.geojson")

# ==== 10) Carregar resultados de volta ao PostGIS ====
gdf_result = joined[['nome_uc', 'geometry']]
gdf_result.to_postgis(
    name='ucs_intersectadas', con=en, schema='deltamap', if_exists='replace', index=False
)
print("Tabela 'ucs_intersectadas' criada no PostGIS.")
